In [10]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import random
import copy
import json

# === Load dataset ===
data = pd.read_fwf("RC104_25.txt", skiprows=8, header=None)
data.columns = ["CUST_NO", "XCOORD", "YCOORD", "DEMAND", "READY_TIME", "DUE_DATE", "SERVICE_TIME"]

data_pre = pd.read_fwf("RC103_25.txt", skiprows=8, header=None)
data_pre.columns = ["CUST_NO", "XCOORD", "YCOORD", "DEMAND", "READY_TIME", "DUE_DATE", "SERVICE_TIME"]

# === Sets ===
V = list(range(25))  # 25 vehicles
C = list(range(1, len(data)))  # Customers
N = list(range(len(data)))  # All nodes

# === Parameters ===
dc = data['DEMAND'].tolist()
dc[0] = 0
dc_pre = data_pre['DEMAND'].tolist()
dc_pre[0] = 0
coords = list(zip(data['XCOORD'], data['YCOORD']))
ai = data['READY_TIME'].tolist()
bi = data['DUE_DATE'].tolist()
si = data['SERVICE_TIME'].tolist()

cij = 30
delta = 0.17
lambda_v = 200
pi = 10
theta = 0.7
mv = 100000
cv = 100
hv = 15
ev = 0.1
cf = 0.5
fv = 0.1
M = 1000000
w1, w2, w3 = 1, 1, 1

# Derived reverse demand
dc_rev = [round(d * delta) for d in dc_pre]

# Distance matrix
t = np.zeros((len(N), len(N)))
for i in N:
    for j in N:
        t[i][j] = math.hypot(coords[i][0] - coords[j][0], coords[i][1] - coords[j][1])


In [14]:
# === Load initial routes ===
with open("routes.json", "r") as f:
    initial_routes = json.load(f)

# Best objective values from exact model
Feco_best = 18390.9967
Fenv_best = 30.4283
Fsoc_best = 411.1106

def evaluate_solution(routes):
    # === Initialize costs and unsatisfied demand ===
    Feco = Fenv = Fsoc = 0
    unsat_delivery = [0] + [dc[i] for i in C]  # forward demand Qd
    unsat_return = [0] + [dc_rev[i] for i in C]  # reverse demand Qr

    for v, route in enumerate(routes):
        if not route or len(route) < 2:
            continue

        time = 0  # total route time βv
        load = 0  # vehicle load Lv
        last = route[0]  # start at depot

        for idx in range(1, len(route)):
            curr = route[idx]
            if curr == 0:
                last = curr
                continue

            dist = t[last][curr]

            # Time window constraints
            arrive_time = time + dist
            start_service = max(arrive_time, ai[curr])  

            if start_service > bi[curr]: 
                return float("inf")  

            time = start_service + si[curr]  

            # Forward and reverse load update
            delivery = min(dc[curr], lambda_v - load)  # Fc_iv forward flow
            return_qty = min(dc_rev[curr], load)  # Rc_iv reverse flow

            load += delivery - return_qty  # Load balance constraint

            if load > lambda_v:  # Capacity constraint
                return float("inf")

            # Update unsatisfied demand quantities Qd and Qr
            unsat_delivery[curr] = max(0, unsat_delivery[curr] - delivery)
            unsat_return[curr] = max(0, unsat_return[curr] - return_qty)

            Feco += dist * (cf * fv + cij)  # transportation + fuel cost
            Fenv += ev * dist  # emission cost
            last = curr

        if time > mv:  # Limit route time 
            return float("inf")

        Feco += cv + hv * time  # fixed + driver wage
        Fsoc += time  # total working time

    # Objective function
    Fsat = sum(theta * unsat_delivery[i] + (1 - theta) * unsat_return[i] for i in C)
    F1 = w1 * (Feco - Feco_best) / Feco_best + \
         w2 * (Fenv - Fenv_best) / Fenv_best + \
         w3 * (Fsoc - Fsoc_best) / Fsoc_best

    return F1 + Fsat

def random_destroy(routes, fraction=0.6):
    # Randomly remove a portion of customers from their assigned routes
    all_customers = [(v, cust) for v in range(len(routes)) for cust in routes[v] if cust != 0]
    num_remove = int(len(all_customers) * fraction)
    to_remove = random.sample(all_customers, num_remove)
    removed = []
    for v, cust in to_remove:
        if cust in routes[v]:
            routes[v].remove(cust)
            removed.append(cust)
    return removed, routes

def greedy_repair(routes, removed):
    # Insert each removed customer into the position that gives best objective value
    for cust in removed:
        best_cost = float("inf")
        best_pos = None
        for v in range(len(routes)):
            for i in range(1, len(routes[v])):
                temp_routes = copy.deepcopy(routes)
                temp_routes[v].insert(i, cust)
                cost = evaluate_solution(temp_routes)
                if cost < best_cost:
                    best_cost = cost
                    best_pos = (v, i)
        if best_pos:
            v, i = best_pos
            routes[v].insert(i, cust)
    return routes

def lns_sa(initial_routes, iterations=1000, initial_temp=200, cooling_rate=0.98, destroy_fraction=0.5):
    
    # === Large Neighborhood Search + Simulated Annealing ===
    current_routes = copy.deepcopy(initial_routes)
    best_routes = copy.deepcopy(current_routes)
    best_cost = evaluate_solution(current_routes)
    temperature = initial_temp

    for it in range(iterations):
        removed, partial_routes = random_destroy(copy.deepcopy(current_routes), fraction=destroy_fraction)
        repaired_routes = greedy_repair(partial_routes, removed)

        new_cost = evaluate_solution(repaired_routes)
        old_cost = evaluate_solution(current_routes)

        delta = new_cost - old_cost
        if delta < 0 or random.random() < math.exp(-delta / temperature):  # SA acceptance criterion
            current_routes = copy.deepcopy(repaired_routes)
            if new_cost < best_cost:
                best_cost = new_cost
                best_routes = copy.deepcopy(repaired_routes)

        temperature *= cooling_rate

        if it % 100 == 0:
            print(f"Iteration {it}: Current = {old_cost:.2f}, Best = {best_cost:.2f}")

    return best_routes, best_cost

# === Run ===
lns_routes, lns_cost = lns_sa(initial_routes)

print("\nFinal LNS + SA Cost:", lns_cost)
for v, route in enumerate(lns_routes):
    if route:
        print(f"Vehicle {v}: {route}")


Iteration 0: Current = 2.86, Best = 2.19
Iteration 100: Current = 262.38, Best = 1.71
Iteration 200: Current = 263.34, Best = 1.71
Iteration 300: Current = 262.47, Best = 1.71
Iteration 400: Current = 262.09, Best = 1.71
Iteration 500: Current = 262.09, Best = 1.71
Iteration 600: Current = 262.09, Best = 1.71
Iteration 700: Current = 262.09, Best = 1.71
Iteration 800: Current = 262.09, Best = 1.71
Iteration 900: Current = 262.09, Best = 1.71

Final LNS + SA Cost: 1.7143588388070565
Vehicle 3: [0, 8, 7, 6, 2, 4, 5, 3, 1]
Vehicle 7: [0, 14, 17, 15, 11, 10, 9, 12, 13, 16]
Vehicle 11: [0, 20, 22, 24, 19, 23, 21, 25, 18]
